In [1]:
from smtutil import hiveutil, oracle_to_hive
import pandas as pd
from pyspark.sql.functions import lag, lead, first, last, udf, lit, col, array_sort, collect_list, expr, row_number, lit,concat
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, StructType, StructField, DoubleType, BooleanType
from pyspark.sql import functions as func
from datetime import datetime as dt
import datetime
import json
from datetime import date, timedelta, datetime
import os
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)
from typing import Union

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

from weblog_util.common import getCleansedDomainList, getAggValue, getDomainSummary, getDomainSummary_v2, getRawDomainSummary

Connect to Oracle Successfully
Connect to Oracle Successfully


In [2]:
# spark.stop()
# conn.close()
# sc.stop()

In [3]:
sc, spark = hiveutil.get_sc_spark()

In [4]:
conn=oracle_to_hive.connect_to_Oracle()

Connect to Oracle Successfully


## Params

In [5]:
val_agg_period_start, val_agg_period_end = 20221016, 20230115
agg_freq = 'daily' # 'weekly', 'monthly'
data_source = 'oracle'
# True if can combine all time_key into one row, i.e. groupby subr num only
gp_by_subr_only = True
get_flag = True

if agg_freq == 'daily':
    t_domain_table='t_domain_daily'
    t_domain_time_key='date_id'
elif agg_freq == 'weekly':
    t_domain_table='t_domain_weekly'
    t_domain_time_key='week_id'
elif agg_freq == 'monthly':
    t_domain_table='t_domain_monthly'
    t_domain_time_key='month_id'

# t_domain_table='t_domain_monthly'
# t_domain_time_key='month_id'

print(f'[Info] Period: {val_agg_period_start} to {val_agg_period_end}')
print(f'[Info] Aggregate Freq: {agg_freq}, {t_domain_table}, {t_domain_time_key}')
print(f'[Info] Source: {data_source}')

[Info] Period: 20221016 to 20230115
[Info] Aggregate Freq: daily, t_domain_daily, date_id
[Info] Source: oracle


## Raw Data

### Customer Base - Prepaid

In [6]:
# df_prepaid_mas_n1 = pd.read_sql_query("""
# select
#                 VW_PREPD_POSTPAID_SUBR_N1.CUST_NUM,
#                 VW_PREPD_POSTPAID_SUBR_N1.SUBR_NUM,
#                 VW_PREPD_POSTPAID_SUBR_N1.D_HKID_N

# from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1

# where 
#     VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type = 'PR'
#     AND VW_PREPD_POSTPAID_SUBR_N1.SUBR_STAT_CD = 'A1'
# """,con=conn)

# # df_prepaid_mas_n1 = pd.read_sql_query("""
# # select
# #                 VW_PREPD_POSTPAID_SUBR_N1.CUST_NUM,
# #                 VW_PREPD_POSTPAID_SUBR_N1.SUBR_NUM,
# #                 VW_PREPD_POSTPAID_SUBR_N1.D_HKID_N

# # from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1
                
# # left join PRD_BIZ_SUMM_VW.VW_RATE_PLAN_REF
# # on VW_PREPD_POSTPAID_SUBR_N1.Rate_Plan_Cd = VW_RATE_PLAN_REF.Rate_Plan_Cd

# # left join PRD_BIZ_SUMM_VW.VW_PREPD_DEMO_CARD
# # on VW_PREPD_POSTPAID_SUBR_N1.Subr_Num = VW_PREPD_DEMO_CARD.Msisdn

# # """,con=conn)
# df_prepaid_mas_n1.head()

### Customer Base - Postpaid

In [7]:
# df_tmp = pd.read_sql('select * from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1 where rownum<=1', con=conn)

In [7]:
# df_postpaid_mas_n1 = pd.read_sql_query("""
# select
#                 VW_PREPD_POSTPAID_SUBR_N1.CUST_NUM,
#                 VW_PREPD_POSTPAID_SUBR_N1.SUBR_NUM,
#                 VW_PREPD_POSTPAID_SUBR_N1.D_HKID_N,
#                 VW_RATE_PLAN_REF.Rate_Plan_Grp AS RATE_PLAN_GRP,
#                 VW_SHK_RATE_PLAN_GRP_REF.SHK_PLAN_SUBGRP AS SHK_PLAN_SUBGRP,
#                 VW_PREPD_POSTPAID_SUBR_N1.D_AGE AS AGE,
#                 VW_PREPD_POSTPAID_SUBR_N1.GENDER

# from PRD_BIZ_SUMM_VW.VW_PREPD_POSTPAID_SUBR_N1
# left join PRD_BIZ_SUMM_VW.VW_RATE_PLAN_REF
# on VW_PREPD_POSTPAID_SUBR_N1.Rate_Plan_Cd = VW_RATE_PLAN_REF.Rate_Plan_Cd

# left join PRD_BIZ_SUMM_VW.VW_PREPD_DEMO_CARD
# on VW_PREPD_POSTPAID_SUBR_N1.Subr_Num = VW_PREPD_DEMO_CARD.Msisdn

# left join PRD_BIZ_SUMM_VW.VW_SUBR_NOMINATION_HIST 
# on VW_PREPD_POSTPAID_SUBR_N1.Cust_Num = VW_SUBR_NOMINATION_HIST.CUST_NUM
# and VW_PREPD_POSTPAID_SUBR_N1.subr_num = VW_SUBR_NOMINATION_HIST.subr_num

# left join PRD_BIZ_SUMM_VW.VW_SUBR_DATA_ENTITLE
# on VW_PREPD_POSTPAID_SUBR_N1.Cust_Num = VW_SUBR_DATA_ENTITLE.CUST_NUM
# and  VW_PREPD_POSTPAID_SUBR_N1.subr_num = VW_SUBR_DATA_ENTITLE.subr_num

# left join PRD_BIZ_SUMM_VW.VW_SHK_RATE_PLAN_GRP_REF
# on VW_SUBR_DATA_ENTITLE.FREE_DATA_ENTITLE = VW_SHK_RATE_PLAN_GRP_REF.FREE_DATA_ENTITLE

# WHERE VW_PREPD_POSTPAID_SUBR_N1.D_Active_Subr_Flg = 'Y'
#                 AND VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type = 'PO'
#                 AND CASE 
#                                 WHEN (
#                                                                 VW_PREPD_POSTPAID_SUBR_N1.Prepost_Type <> 'PO'
#                                                                 AND VW_PREPD_DEMO_CARD.Msisdn <> ' '
#                                                                 )
#                                                 THEN 'N'
#                                 WHEN (
#                                                                 (coalesce(VW_RATE_PLAN_REF.Rate_Plan_Grp, ' ')) IN (
#                                                                                 'Add-on Numbers'
#                                                                                 ,'Non Revenue Mobile Plan'
#                                                                                 ,'Non Revenue BB Line'
#                                                                                 ,'DHL Special Project'
#                                                                                 )
#                                                                 )
#                                                 THEN 'N'
#                                 WHEN (
#                                                                 (coalesce(VW_PREPD_POSTPAID_SUBR_N1.Cust_Type_Cd, ' ')) IN (
#                                                                                 'SMC'
#                                                                                 ,'LPUC1'
#                                                                                 ,'HKBU1'
#                                                                                 ,'RWSU1'
#                                                                                 ,'HKBU2'
#                                                                                 ,'GCU1'
#                                                                                 )
#                                                                 )
#                                                 THEN 'N'
#                                 ELSE 'Y'
#                                 END = 'Y'
#                 AND VW_PREPD_POSTPAID_SUBR_N1.NC_FLG = 'N'
#                 AND VW_SHK_RATE_PLAN_GRP_REF.SHK_PLAN_SUBGRP NOT IN (
#                                 'Excluded'
#                                 ,'MVNOs (Local)'
#                                 ,'MVNOs (Roaming)'
#                                 )
#                 AND VW_SUBR_NOMINATION_HIST.NOMINATION_FLG = 'N'
# """,con=conn)

In [8]:
# df_postpaid_mas_n1.shape

In [9]:
# # sdf = sdf_9mth_up.drop_duplicates(['HKID_BR'])
# # print(sdf.count())
# sdf = spark.createDataFrame(df_postpaid_mas_n1.drop_duplicates(['SUBR_NUM']))
# print(sdf.count())

In [10]:
# sdf.limit(2).toPandas()

## Filter Table

#### Activation Channel

In [12]:
# print(datetime.now())
# df_act_ch = pd.read_sql('''
#     select 
#         SUBR_NUM, 
#         CHANNEL_EFFORT,
#         row_number() over (partition by SUBR_NUM order by INV_DATE asc) RANK
#     from
#         PRD_BIZ_SUMM_VW.VW_RBD_ALL_CHANNEL_ACT_RE_TX_U
#     ''', con=conn)
# print(datetime.now())
# # select SUBR_NUM, Channel_Effort PRD_BIZ_SUMM_VW.VW_RBD_ALL_CHANNEL_ACT_RE_TX_U
# # (need min the INV_DATE)

# # # to csv
# df_act_ch.to_csv('./activate_channel.csv', index=False)

# sdf_act_ch = spark.read.option("header",True).csv('./activate_channel.csv')
# sdf_act_ch = sdf_act_ch.filter(F.col('RANK')==1).drop('RANK').cache()
# sdf_act_ch.limit(5).toPandas()

# # join with customer base
# sdf_filter_table = sdf.join(sdf_act_ch, 'SUBR_NUM', 'left').drop_duplicates(['D_HKID_N']).drop_duplicates(['SUBR_NUM'])

In [13]:
# windowSpec_eld  = Window.partitionBy("SUBR_NUM").orderBy(F.col("INV_DATE").asc())
# sdf_act_ch = sdf_act_ch.withColumn("ROW_NUMBER", F.row_number().over(windowSpec_eld))

#### CSV

In [14]:
# # source_table = pd.read_csv('./game_and_music_and_football_lover_with_membership_gender.csv')
# source_table = pd.read_csv('./game_and_music_and_football_lover_with_membership_gender.csv', converters = {'SUBR_NUM': str})

# # PREPROCESSING
# source_table = source_table[source_table['SEGMENT_TYPE']=='Narrowband']
# source_table['CUST_NUM'] = source_table['CUST_NUM'].apply(lambda x: '0' * (8 - len(str(x))) + str(x) if (8 - len(str(x))) > 0 else str(x))

# filter_table = spark.createDataFrame(source_table[['CUST_NUM', 'SUBR_NUM']])

In [11]:
# filter_table
# sdf_filter_table = sdf
# 'NO_OF_LINE_WITH_CE_9MTH_UP')
# F.col('ce_month').alias('CE_MONTH'))

# print(sdf_filter_table.count())
# sdf_filter_table.limit(4).toPandas()

In [12]:
# sdf_filter_table.printSchema()

## End of Customer Base

### t_domain_dictionary

In [6]:
sdf_t_domain = spark.read.parquet('/app/ws-ic/hive/weblog.db/t_domain_dictionary/')
print(sdf_t_domain.count())
sdf_t_domain.limit(5).toPandas()

967868


tag_gp             tag                       domain agg_level  \
0  <Unclassified>  <Unclassified>               WWW.GOOGLE.COM    domain   
1  <Unclassified>  <Unclassified>          PLAY.GOOGLEAPIS.COM    domain   
2  <Unclassified>  <Unclassified>  GOOGLEADS.G.DOUBLECLICK.NET    domain   
3  <Unclassified>  <Unclassified>           WWW.GOOGLEAPIS.COM    domain   
4  <Unclassified>  <Unclassified>          APP-MEASUREMENT.COM    domain   

   agg_value  
0          1  
1          2  
2          3  
3          4  
4          6

## Domain Lists

In [7]:
# l_immigrate = [
#     'WWW.HIGHSPEED.MTR.COM.HK',
#     'WWW.12306.CN',
#     'WWW.QUOTABOOKING.GOV.HK',
#     'WWW.CTRIP.COM',
# ]
l_immigrate = pd.read_excel('/app/ws-simon/data/immigration_list.xlsx')

# clean domain list
l_immigrate = getCleansedDomainList(l_immigrate['domain'])
l_immigrate

['WWW.IMMIGRANT.COM.TW',
 'WWW.TECOS.ORG.HK',
 'WWW.OVERSEAS.EDU.TW',
 'WWW.CHT.COM.TW',
 'WWW.FAREASTONE.COM.TW',
 'WWW.TAIWANMOBILE.COM',
 'WWW.TSTARTEL.COM',
 'WWW.CHIAOSHUN.COM',
 'HONGKONG.CHINA.EMBASSY.GOV.AU',
 'WWW.ABF.GOV.AU',
 'WWW.VFSGLOBAL.COM',
 'IMMI.HOMEAFFAIRS.GOV.AU',
 'WWW.VODAFONE.COM.AU',
 'WWW.OPTUS.COM.AU',
 'WWW.IDP.COM',
 'ASTON.EDU.HK',
 'WWW.DADI.COM.HK',
 'WWW.HONGKONG.GC.CA',
 'WWW.CANADA.CA',
 'CANADIAN-IMMIGRATION.CA',
 'WWW.IMMIGRATION-QUEBEC.GOUV.QC.CA',
 'ICCRC-CRCIC.CA',
 'WWW.INVESTCANADA.CA',
 'WWW.BGCOMMUNICATIONS.CA',
 'WWW.ROGERS.COM',
 'WWW.FREEDOMMOBILE.CA',
 'WWW.TELUSMOBILITY.COM',
 'HIHOSTELS.CA',
 'WWW.EF.COM.HK',
 'WWW.BELL.CA',
 'WWW.TELUS.COM',
 'WWW.ICA.GOV.SG',
 'WWW.GUIDEMESINGAPORE.COM',
 'WWW.MFA.GOV.SG',
 'WWW.PROPERTYGURU.COM.SG',
 'WWW.STPROPERTY.SG',
 'WWW.MOE.GOV.SG',
 'WWW.SINGAPOREEXPATS.COM',
 'WWW.SINGTEL.COM',
 'WWW.STARHUB.COM',
 'WWW.M1.COM.SG',
 'WWW.RIGHTMOVE.CO.UK',
 'WWW.ZOOPLA.CO.UK',
 'WWW.SPAREROOM.CO.UK',
 'WWW.ON

In [8]:
# # search
# domain_search_list = l_immigrate

# doamin_like = ''

# for domain in domain_search_list:
#     print(sdf_t_domain.filter(F.col('domain')==domain).toPandas())
# # sdf_t_domain.filter(F.col('domain').like(doamin_like))

## Get Summary

In [9]:
# domain list , app list, result list
domain_lists = [l_immigrate]
app_name_list = ['IMMIGRATE']

summary_list = []

In [14]:
# # summary = getRawDomainSummary(l_china_traveller, val_agg_period_start, val_agg_period_end, conn)
# print(summary.shape)
# summary.head(1)
# summary.groupby(['SUBR_NUM', 'ACCESS_DATE', 'SSL_CERT_DOMAIN'])['SUBR_NUM'].count().max()

# summary['APPS'] = a
# # rename
# summary.rename(columns={'BYTE_COUNT': 'TOTAL_BYTE_COUNT'}, inplace=True)
# summary['month_id'] = summary['ACCESS_DATE'].dt.strftime('%Y%m')
# summary['date_id'] = summary['ACCESS_DATE'].dt.strftime('%Y%m%d')
# summary['week_id'] = summary['ACCESS_DATE'].dt.strftime('%Y-%U')
# summary.head()

# summary2 = getDomainSummary_v2(agg_value_table=getAggValue(l_china_traveller),
#                               agg_freq=agg_freq,
#                               agg_period_start=val_agg_period_start,
#                               agg_period_end=val_agg_period_end,
#                               app_name='CHINA_TRAVELLER',
#                               filter_table=sdf_filter_table).cache()

In [81]:
# df = summary[['SUBR_NUM', t_domain_time_key, 'TOTAL_BYTE_COUNT', 'TOTAL_HIT', 'APPS']]

In [13]:
# import cx_Oracle
# def getRawDomainSummary_v2(
#     domain_list:list, 
#     t_domain_time_key:str,
#     val_agg_period_start: int, 
#     val_agg_period_end:int, 
#     app_name:str,
#     conn:cx_Oracle.Connection,
#     all_cols:bool=False,
# ):

#     print(f'[Info] Time Start: {datetime.now()}')

#     # format the string
#     search_string_domain = ','.join([f"'{d}'" for d in getCleansedDomainList(domain_list)])

#     search_string = f"""
#     select 
#         /* +PARALLEL(16) */ *
#     from 
#         PRD_BIZ_SUMM_VW.VW_SSL_SED_CALL_SUMM
#     where
#         SSL_CERT_DOMAIN IN ({search_string_domain})
#         and ACCESS_DATE between TO_DATE({val_agg_period_start}, 'yyyymmdd') and TO_DATE({val_agg_period_end}, 'yyyymmdd')
#     """
#     print(search_string)

#     # get raw weblog data from oracle
#     df = pd.read_sql(search_string,con=conn)

#     print(f'[Info] Time End: {datetime.now()}')

#     # add columns
#     df['APPS'] = app_name
#     # rename columns
#     df.rename(columns={'BYTE_COUNT': 'TOTAL_BYTE_COUNT'}, inplace=True)

#     # date/week/month id
#     if t_domain_time_key=='date_id':
#         df['date_id'] = df['ACCESS_DATE'].dt.strftime('%Y%m%d')
#     elif t_domain_time_key=='week_id':
#         df['week_id'] = df['ACCESS_DATE'].dt.strftime('%Y-%U')
#     elif t_domain_time_key=='month_id':
#         df['month_id'] = df['ACCESS_DATE'].dt.strftime('%Y%m')

#     if not all_cols:
#         df = df[['SUBR_NUM', t_domain_time_key, 'TOTAL_BYTE_COUNT', 'TOTAL_HIT', 'APPS']]

#     return df

In [10]:
domain_list = domain_lists[0]
# format the string
search_string_domain = ','.join([f"'{d}'" for d in getCleansedDomainList(domain_list)])
search_string_domain

"'WWW.IMMIGRANT.COM.TW','WWW.TECOS.ORG.HK','WWW.OVERSEAS.EDU.TW','WWW.CHT.COM.TW','WWW.FAREASTONE.COM.TW','WWW.TAIWANMOBILE.COM','WWW.TSTARTEL.COM','WWW.CHIAOSHUN.COM','HONGKONG.CHINA.EMBASSY.GOV.AU','WWW.ABF.GOV.AU','WWW.VFSGLOBAL.COM','IMMI.HOMEAFFAIRS.GOV.AU','WWW.VODAFONE.COM.AU','WWW.OPTUS.COM.AU','WWW.IDP.COM','ASTON.EDU.HK','WWW.DADI.COM.HK','WWW.HONGKONG.GC.CA','WWW.CANADA.CA','CANADIAN-IMMIGRATION.CA','WWW.IMMIGRATION-QUEBEC.GOUV.QC.CA','ICCRC-CRCIC.CA','WWW.INVESTCANADA.CA','WWW.BGCOMMUNICATIONS.CA','WWW.ROGERS.COM','WWW.FREEDOMMOBILE.CA','WWW.TELUSMOBILITY.COM','HIHOSTELS.CA','WWW.EF.COM.HK','WWW.BELL.CA','WWW.TELUS.COM','WWW.ICA.GOV.SG','WWW.GUIDEMESINGAPORE.COM','WWW.MFA.GOV.SG','WWW.PROPERTYGURU.COM.SG','WWW.STPROPERTY.SG','WWW.MOE.GOV.SG','WWW.SINGAPOREEXPATS.COM','WWW.SINGTEL.COM','WWW.STARHUB.COM','WWW.M1.COM.SG','WWW.RIGHTMOVE.CO.UK','WWW.ZOOPLA.CO.UK','WWW.SPAREROOM.CO.UK','WWW.ONTHEMARKET.COM','WWW.BRITISHUNITED.NET','STUDYOVERSEAS.COM.HK','WWW.UKMAJESTY.COM','WWW.B

In [11]:
search_string = f"""
select 
    /* +PARALLEL(16) */ SUBR_NUM, SSL_CERT_DOMAIN
from 
    PRD_BIZ_SUMM_VW.VW_SSL_SED_CALL_SUMM
where
    SSL_CERT_DOMAIN IN ({search_string_domain})
    and ACCESS_DATE between TO_DATE({val_agg_period_start}, 'yyyymmdd') and TO_DATE({val_agg_period_end}, 'yyyymmdd')
group by
    SUBR_NUM, SSL_CERT_DOMAIN
"""
print(search_string)


select 
    /* +PARALLEL(16) */ SUBR_NUM, SSL_CERT_DOMAIN
from 
    PRD_BIZ_SUMM_VW.VW_SSL_SED_CALL_SUMM
where
    SSL_CERT_DOMAIN IN ('WWW.IMMIGRANT.COM.TW','WWW.TECOS.ORG.HK','WWW.OVERSEAS.EDU.TW','WWW.CHT.COM.TW','WWW.FAREASTONE.COM.TW','WWW.TAIWANMOBILE.COM','WWW.TSTARTEL.COM','WWW.CHIAOSHUN.COM','HONGKONG.CHINA.EMBASSY.GOV.AU','WWW.ABF.GOV.AU','WWW.VFSGLOBAL.COM','IMMI.HOMEAFFAIRS.GOV.AU','WWW.VODAFONE.COM.AU','WWW.OPTUS.COM.AU','WWW.IDP.COM','ASTON.EDU.HK','WWW.DADI.COM.HK','WWW.HONGKONG.GC.CA','WWW.CANADA.CA','CANADIAN-IMMIGRATION.CA','WWW.IMMIGRATION-QUEBEC.GOUV.QC.CA','ICCRC-CRCIC.CA','WWW.INVESTCANADA.CA','WWW.BGCOMMUNICATIONS.CA','WWW.ROGERS.COM','WWW.FREEDOMMOBILE.CA','WWW.TELUSMOBILITY.COM','HIHOSTELS.CA','WWW.EF.COM.HK','WWW.BELL.CA','WWW.TELUS.COM','WWW.ICA.GOV.SG','WWW.GUIDEMESINGAPORE.COM','WWW.MFA.GOV.SG','WWW.PROPERTYGURU.COM.SG','WWW.STPROPERTY.SG','WWW.MOE.GOV.SG','WWW.SINGAPOREEXPATS.COM','WWW.SINGTEL.COM','WWW.STARHUB.COM','WWW.M1.COM.SG','WWW.RIGHTMOVE.CO.UK','

In [12]:
df_summary = pd.read_sql(search_string ,con=conn)

In [13]:
df_summary.to_csv('/app/ws-simon/data/immigrate_list_20230116_v2.csv', index=False)

In [20]:
df_summary.drop_duplicates('SUBR_NUM').to_csv('/app/ws-simon/data/immigrate_list_20230117.csv', index=False)

In [42]:
# ### domain list, app name list
# for dl, a in zip(domain_lists, app_name_list):

#     if data_source == 'oracle' and agg_freq == 'daily':
#         print('[Info] Get raw data from Oracle:' )
#         df_summary = getRawDomainSummary(
#             domain_list=dl,
#             t_domain_time_key=t_domain_time_key,
#             val_agg_period_start=val_agg_period_start, 
#             val_agg_period_end=val_agg_period_end, 
#             app_name=a,
#             conn=conn,
#             all_cols=False,
#         )
#         print(df_summary.head())
#         summary = spark.createDataFrame(df_summary)
#     else:
#         print('[Info] Get data from t_domain:' )
#         summary = getDomainSummary_v2(agg_value_table=getAggValue(dl),
#                                       agg_freq=agg_freq,
#                                       agg_period_start=val_agg_period_start,
#                                       agg_period_end=val_agg_period_end,
#                                       app_name=a,
#                                       filter_table=sdf_filter_table).cache()
    
#         print(summary.limit(5).toPandas())
    
#     # add to list
#     summary_list.append(summary)
#     print(f'[Info] Finish summary *{a}*')

[Info] Get raw data from Oracle:
[Info] Time Start: 2023-01-16 15:12:07.206454

    select 
        /* +PARALLEL(16) */ *
    from 
        PRD_BIZ_SUMM_VW.VW_SSL_SED_CALL_SUMM
    where
        SSL_CERT_DOMAIN IN ('WWW.IMMIGRANT.COM.TW','WWW.TECOS.ORG.HK','WWW.OVERSEAS.EDU.TW','WWW.CHT.COM.TW','WWW.FAREASTONE.COM.TW','WWW.TAIWANMOBILE.COM','WWW.TSTARTEL.COM','WWW.CHIAOSHUN.COM','HONGKONG.CHINA.EMBASSY.GOV.AU','WWW.ABF.GOV.AU','WWW.VFSGLOBAL.COM','IMMI.HOMEAFFAIRS.GOV.AU','WWW.VODAFONE.COM.AU','WWW.OPTUS.COM.AU','WWW.IDP.COM','ASTON.EDU.HK','WWW.DADI.COM.HK','WWW.HONGKONG.GC.CA','WWW.CANADA.CA','CANADIAN-IMMIGRATION.CA','WWW.IMMIGRATION-QUEBEC.GOUV.QC.CA','ICCRC-CRCIC.CA','WWW.INVESTCANADA.CA','WWW.BGCOMMUNICATIONS.CA','WWW.ROGERS.COM','WWW.FREEDOMMOBILE.CA','WWW.TELUSMOBILITY.COM','HIHOSTELS.CA','WWW.EF.COM.HK','WWW.BELL.CA','WWW.TELUS.COM','WWW.ICA.GOV.SG','WWW.GUIDEMESINGAPORE.COM','WWW.MFA.GOV.SG','WWW.PROPERTYGURU.COM.SG','WWW.STPROPERTY.SG','WWW.MOE.GOV.SG','WWW.SINGAPOREEXPATS.C

KeyboardInterrupt: 

## Pivot & Merge

In [32]:
# merge all summaries
res_merged = summary_list[0]

if len(summary_list) > 1:
    for summary in summary_list[1:]:
        res_merged = res_merged.union(summary).cache()

In [33]:
print(res_merged.count())
res_merged.limit(2).show()

50885
+----------+--------+----------------+---------+---------------+
|  SUBR_NUM| date_id|TOTAL_BYTE_COUNT|TOTAL_HIT|           APPS|
+----------+--------+----------------+---------+---------------+
|1994411010|20221211|         11429.0|        1|CHINA_TRAVELLER|
|  54016153|20221211|          9499.0|        1|CHINA_TRAVELLER|
+----------+--------+----------------+---------+---------------+



In [34]:
# merge
if gp_by_subr_only:
    res_pivot=res_merged.groupBy('SUBR_NUM')\
                        .pivot('APPS').agg(F.sum('TOTAL_BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),\
                                           F.sum('TOTAL_HIT').alias('TOTAL_HIT'),\
                                          ).na.fill(0).withColumnRenamed(t_domain_time_key, t_domain_time_key.upper())
else:
    res_pivot=res_merged.groupBy(t_domain_time_key, 'SUBR_NUM')\
                        .pivot('APPS').agg(F.sum('TOTAL_BYTE_COUNT').alias('TOTAL_BYTE_COUNT'),\
                                           F.sum('TOTAL_HIT').alias('TOTAL_HIT'),\
                                          ).na.fill(0).withColumnRenamed('month_id','MONTH_ID')

In [35]:
print(res_pivot.count())
res_pivot.limit(2).toPandas()

36292


SUBR_NUM  CHINA_TRAVELLER_TOTAL_BYTE_COUNT  CHINA_TRAVELLER_TOTAL_HIT
0  96024518                           13829.0                          1
1  67005259                         7386797.0                         20

#### Refine Customer Base

In [36]:
sdf.limit(1).toPandas()

CUST_NUM  SUBR_NUM    D_HKID_N                            RATE_PLAN_GRP  \
0  00069109  90190173  G500463(1)  Smartphone voice & data sharing (SIM)-L   

                               SHK_PLAN_SUBGRP  AGE GENDER  
0  Family plans (Main SIM) without add-on pack   58      M

In [38]:
sdf_to_join = sdf
# sdf_to_join = sdf.select(
#     'CUST_NUM',
#     'SUBR_NUM',
#     'CURRENT_SHK_PLAN_GRP',
#     'CURRENT_SHK_PLAN_SUBGRP',
#      F.col('ce_month').alias('CE_MONTH'),
# )

### joining

In [39]:
sdf_output = sdf_to_join.join(res_pivot, 'SUBR_NUM', 'left').na.fill(0)

In [40]:
print(sdf_output.count())
sdf_output.limit(2).toPandas()

1035892


SUBR_NUM  CUST_NUM    D_HKID_N    RATE_PLAN_GRP  \
0  46252242  12504208  D474125(1)    5G Add-on SIM   
1  46267377  07403563  K009595(8)  4.5G Add-on SIM   

                                   SHK_PLAN_SUBGRP  AGE GENDER  \
0                                    5G Add-on SIM   62      F   
1  SuperCare Add-on SIM (Secondary SIM of 5G plan)   50      M   

   CHINA_TRAVELLER_TOTAL_BYTE_COUNT  CHINA_TRAVELLER_TOTAL_HIT  
0                               0.0                          0  
1                               0.0                          0

In [41]:
sdf_output.describe().toPandas()

summary             SUBR_NUM           CUST_NUM              D_HKID_N  \
0   count              1035892            1035892               1035892   
1    mean  8.224510077560402E7   9432123.02378916  3.156177677313797E14   
2  stddev  1.642958732842694E7  3009293.538801359  1.152235258971086E16   
3     min             44003911           00000044              00002604   
4     max             98899965           12900518            z858429?0?   

       RATE_PLAN_GRP                       SHK_PLAN_SUBGRP  \
0            1035892                               1035892   
1               None                                  None   
2               None                                  None   
3  21Mbps Add-on SIM           "True" unlimited data plans   
4     other SmarTone  Upper low tier plans (500MB - 1.4GB)   

                   AGE   GENDER CHINA_TRAVELLER_TOTAL_BYTE_COUNT  \
0              1035892  1035892                          1035892   
1  -40.595610353202844     None               37069.737666667956   
2   290.54710176754526     None               467019.84048544004   
3                -6174        C                              0.0   
4                  108        U                     1.08212897E8   

  CHINA_TRAVELLER_TOTAL_HIT  
0                   1035892  
1       0.13465786008580044  
2         1.628599531347067  
3                         0  
4                       313

### Columns Rename

In [42]:
# sdf_output = sdf_output.withColumnRenamed('PARENT UTILITY APPS', 'PARENT_UTILITY_APPS')
sdf_output_final = sdf_output.alias('1')

if get_flag:
    for cat in app_name_list:
        sdf_output_final = sdf_output_final.withColumn(f'{cat}_FLAG', F.when(F.col(f'{cat}_TOTAL_BYTE_COUNT')>0, 1).otherwise(0))

# sdf_output_final = sdf_output.withColumn('ECLASS_FLAG', F.when(F.col('ECLASS')>0, 1).otherwise(0)) \
#                              .withColumn('EDUCATION_RESOURCE_WEB_FLAG', F.when(F.col('EDUCATION_RESOURCE_WEB')>0, 1).otherwise(0)) \
#                              .withColumn('ENTERTAINMENT_FLAG', F.when(F.col('ENTERTAINMENT')>0, 1).otherwise(0)) \
#                              .withColumn('KIDS_TOP100_DOMAINS_FLAG', F.when(F.col('KIDS_TOP100')>0, 1).otherwise(0)) \
#                              .withColumn('PARENT_UTILITY_APPS_FLAG', F.when(F.col('PARENT_UTILITY_APPS')>0, 1).otherwise(0)) \
#                              .withColumn('SCHOOl_WEB_FLAG', F.when(F.col('SCHOOl_WEB')>0, 1).otherwise(0)) \

In [43]:
print(sdf_output_final.count(), len(sdf_output_final.columns))
sdf_output_final.limit(2).toPandas()

1035892 10


SUBR_NUM  CUST_NUM    D_HKID_N    RATE_PLAN_GRP  \
0  46252242  12504208  D474125(1)    5G Add-on SIM   
1  46267377  07403563  K009595(8)  4.5G Add-on SIM   

                                   SHK_PLAN_SUBGRP  AGE GENDER  \
0                                    5G Add-on SIM   62      F   
1  SuperCare Add-on SIM (Secondary SIM of 5G plan)   50      M   

   CHINA_TRAVELLER_TOTAL_BYTE_COUNT  CHINA_TRAVELLER_TOTAL_HIT  \
0                               0.0                          0   
1                               0.0                          0   

   CHINA_TRAVELLER_FLAG  
0                     0  
1                     0

In [44]:
# # select subcolumns
# sdf_output_final = sdf_output_final.select(
#     'SUBR_NUM',
#     'CUST_NUM',
#     'CURRENT_SHK_PLAN_GRP',
#     'CURRENT_SHK_PLAN_SUBGRP',
#     'CE_MONTH',
#     'ECLASS_FLAG',
#     'EDUCATION_RESOURCE_WEB_FLAG',
#     'ENTERTAINMENT_FLAG',
#     'KIDS_TOP100_DOMAINS_FLAG',
#     'PARENT_UTILITY_APPS_FLAG',
#     'SCHOOl_WEB_FLAG',
# )

### Output to csv

In [45]:
df_summary = sdf_output_final.toPandas()

In [46]:
time_str = datetime.strftime(datetime.now(), '%Y%m%d%H%M%S')

In [47]:
df_summary.to_csv(f"./output/target_user_by_weblog_{time_str}.csv", index=False)
# save to parquet
# summary.write.mode('append').parquet(f'/app/ws-simon/projects/football_lover_20221111/data/')

In [48]:
sdf_output_final.describe().toPandas()

summary             SUBR_NUM           CUST_NUM              D_HKID_N  \
0   count              1035892            1035892               1035892   
1    mean  8.224510077560402E7   9432123.02378916  3.156177677313797E14   
2  stddev  1.642958732842694E7  3009293.538801359  1.152235258971086E16   
3     min             44003911           00000044              00002604   
4     max             98899965           12900518            z858429?0?   

       RATE_PLAN_GRP                       SHK_PLAN_SUBGRP  \
0            1035892                               1035892   
1               None                                  None   
2               None                                  None   
3  21Mbps Add-on SIM           "True" unlimited data plans   
4     other SmarTone  Upper low tier plans (500MB - 1.4GB)   

                   AGE   GENDER CHINA_TRAVELLER_TOTAL_BYTE_COUNT  \
0              1035892  1035892                          1035892   
1  -40.595610353202844     None               37069.737666667956   
2   290.54710176754526     None               467019.84048544004   
3                -6174        C                              0.0   
4                  108        U                     1.08212897E8   

  CHINA_TRAVELLER_TOTAL_HIT  CHINA_TRAVELLER_FLAG  
0                   1035892               1035892  
1       0.13465786008580044  0.022357543064334892  
2         1.628599531347067   0.14784351332686627  
3                         0                     0  
4                       313                     1

In [49]:
pd.DataFrame(df_summary[[x for x in df_summary.columns if '_FLAG' in x ]].sum()).to_csv(f"./output/target_user_by_weblog_summary_{time_str}.csv")

## End

In [ ]:
# df = df_summary[df_summary['TOTAL_BYTE_COUNT'] >= ]